In [1]:
## StyleGAN 깃허브 저장소 다운로드 링크 :
# https://github.com/rosinality/stylegan2-pytorch/archive/refs/heads/master.zip

In [ ]:
## StyleGAN 256x256 해상도 학습 모델 다운로드 링크
# https://drive.google.com/file/d/1fOeBiMnda5bRzxkBedidY_bYv79t96D1/view?usp=drive_link

In [ ]:
import os
os.chdir('stylegan2-pytorch-master')

import argparse
import torch
from torchvision import utils
from tqdm import tqdm
from model import Generator

def generate(g_ema, device, mean_latent, pics, sample, latent, truncation, mean_latent):
    with torch.no_grad():
        g_ema.eval()
        for i in tqdm(range(pics)):
            sample_z = torch.randn(sample, latent, device=device)

            sample, _ = g_ema(
                [sample_z], truncation=truncation, truncation_latent=mean_latent
            )

            utils.save_image(
                sample,
                f"sample/{str(i).zfill(6)}.png",
                nrow=1,
                normalize=True,
                range=(-1, 1),
            )

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
img_size = 256
sample = 3  
pics = 5
truncation = 1
truncation_mean = 4096
ckpt_path = 'StyleGAN2.pt'
channel_multiplier = 2
latent = 512
n_mlp = 8

g_ema = Generator(img_size, latent, n_mlp, 
                  channel_multiplier=channel_multiplier).to(device)
checkpoint = torch.load(ckpt_path)

g_ema.load_state_dict(checkpoint["g_ema"], strict=False)

if truncation < 1:
    with torch.no_grad():
        mean_latent = g_ema.mean_latent(truncation_mean)
else:
    mean_latent = None

generate(g_ema, device, mean_latent, pics, sample, latent, truncation)